In [7]:
from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd

from ta.trend import ema_indicator
from ta.momentum import stochrsi, rsi
from ta.volume import volume_weighted_average_price

from modules.backtester import long_only_backtester
from modules.data_fetcher import download_historical_data


In [13]:
symbol = "BTC-USDT"
vwap_window = 19
rsi_window = 25

df_BTC = download_historical_data(symbol, "1hour").loc["2021-11-20":]
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()


(11315, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2021-11-20 00:00:00,1.637363e+09,57814.4,58048.5,58134.5,57724.6,185.514622,1.074426e+07
2021-11-20 01:00:00,1.637366e+09,58049.2,58249.8,58574.0,57943.0,343.029016,2.000276e+07
2021-11-20 02:00:00,1.637370e+09,58252.4,58804.8,58871.4,58182.1,385.548711,2.262044e+07
2021-11-20 03:00:00,1.637374e+09,58804.7,58789.5,58969.8,58738.6,191.554901,1.127261e+07
2021-11-20 04:00:00,1.637377e+09,58789.4,58518.7,58842.8,58447.5,262.897002,1.542063e+07


# Standard RSI


In [14]:
df_BTC["RSI_VWAP"] = rsi(
    volume_weighted_average_price(
        df_BTC.High, df_BTC.Low, df_BTC.Close, df_BTC["Volume"], vwap_window
    ),
    rsi_window,
)
df_BTC['EMA80'] = ema_indicator(df_BTC.Close,80)
df_BTC.dropna(inplace=True)
df_BTC.head(15)


,Timestamp,Open,Close,High,Low,Amount,Volume,RSI_VWAP,EMA80
Date,,,,,,,,,
2021-11-23 07:00:00,1.637647e+09,56548.3,56375.7,56631.8,56205.1,238.117694,1.343906e+07,10.756632,57810.969004
2021-11-23 08:00:00,1.637651e+09,56375.7,56018.8,56392.9,55839.0,304.196759,1.705388e+07,10.287668,57766.717918
2021-11-23 09:00:00,1.637654e+09,56010.8,56262.0,56388.7,55705.0,353.442985,1.979990e+07,8.797212,57729.564389
2021-11-23 10:00:00,1.637658e+09,56271.6,56619.8,57300.0,55320.0,664.160843,3.744037e+07,8.031258,57702.162799
2021-11-23 11:00:00,1.637662e+09,56619.8,56336.2,57849.0,56267.8,557.681368,3.180160e+07,7.777736,57668.435322
2021-11-23 12:00:00,1.637665e+09,56342.9,56138.2,56563.7,55951.8,302.225884,1.699804e+07,7.315978,57630.651734
2021-11-23 13:00:00,1.637669e+09,56139.8,56307.1,56439.2,56103.9,248.499223,1.397984e+07,7.083722,57597.971445
2021-11-23 14:00:00,1.637672e+09,56307.1,56789.1,56918.6,56307.1,311.849131,1.768325e+07,7.022401,57577.999310
2021-11-23 15:00:00,1.637676e+09,56789.0,57693.4,57696.1,56574.6,636.405049,3.629697e+07,12.659026,57580.848710


In [15]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.RSI_VWAP > 19 and prev_row.RSI_VWAP < 19 and row.EMA80 < row.Close else False


def sell_func(row: pd.Series, prev_row: pd.Series, trading_days: int) -> bool:
    return True if row.RSI_VWAP < 80 and prev_row.RSI_VWAP > 80 else False


long_only_backtester(df_BTC, buy_func, sell_func)

-------------  General informations  -------------
Period: [2021-11-23 07:00:00] -> [2023-03-06 10:00:00]
Intial balance: 1000 $

-------------  Strategy performance  -------------
Final balance: 1236.79 $
Final net balance: 1191.04 $
Strategy net return: 119.10 %
Buy and Hold return: 39.74 %
Strategy winrate: 57.89 %
Strategy fees: 41.23 $

-------------  Trades informations  --------------
Mean trade duration: 5 days 03:03:09
Total trades: 19
Total good trades: 11
Mean good trades return: 5.48 %
Median good trades return: 3.23 %
Mean good trade duration: 3 days 18:00:00
Total bad trades: 8
Mean bad trades return: -4.37 %
Median bad trades return: -3.12 %
Mean bad trade duration: 7 days 00:30:00
Exit reasons repartition: 
- Exit position triggered: 19
